### 引入库

In [1]:
import pdfplumber
import pandas as pd
import numpy as np

### 扫描PDF，文本转存TXT
##### 预计同转存EXCEL模块合并为一个功能
##### 创建按钮时可以添加另存为功能？自己选择地址

In [4]:
##PDF文档存储地址
# pdfPath = "./failure_code_table_test.pdf"
pdfPath = "./S120_failure_code_table.pdf"

##依次扫描PDF页面并串联文本
pageText = ''
with pdfplumber.open(pdfPath) as pdf:
    for n in range(len(pdf.pages)):
        singlePage = pdf.pages[n]
        singlePageText = singlePage.extract_text()
        pageText = pageText + singlePageText

##完整文本转存TXT格式
txtPath = './S120_failure_code_table.txt'
with open(txtPath, 'w', encoding = 'utf-8') as c:
    c.write(pageText)
    c.close()

print("Done!")

Done!


### 从TXT提取关键信息并转存EXCEL（脑残版）
##### 作为一个小功能设额外按钮（虽然不一定用到）
##### 这一模块可直接作为一个py文件调用函数CreateExcel

In [8]:
txtPath = './S120_failure_code_table.txt'
## 获取每行的信息和内容
with open(txtPath, 'r', encoding = 'utf-8') as f:
    allLine = f.readlines()
    f.close()
allLineNumber = len(allLine)
# print(allLineNumber)

## 剔除不需要的信息行
with open(txtPath,"w",encoding="utf-8") as f_w:
    for line in allLine:
        if "故障和报警" in line:
            continue
        if "SINAMICS S120/S150" in line:
            continue
        if "参数手册," in line:
            continue
        f_w.write(line)

## 刷新每行的信息和内容
with open(txtPath, 'r', encoding = 'utf-8') as f:
    allLine = f.readlines()
    f.close()
allLineNumber = len(allLine)
# print(allLineNumber)

## 筛选分类文本信息
failure = {}                #故障码和名称
failureNumber = 0
failureLocation = {}

informationValue = {}           #信息值
informationValueNumber = 0
informationValueLocation = {}

informationCatefory = {}        #信息类别
informationCateforyNumber = 0
informationCateforyLocation = {}

drivingObject = {}
drivingObjectNumber = 0         #驱动对象数量
drivingObjectLocation = {}    #驱动对象所在首行

component = {}
componentNumber = 0             #组件数量 
componentLocation = {}        #组件所在行

reason = {}
reasonNumber = 0                #原因数量
reasonLocation = {}           #原因所在首行

processing = {}
processingNumber = 0            #处理数量
processingLocation = {}       #处理所在首行

########################
###基于面向对象思想编写###
########################
class Foo:
    def __init__(self, search, name, number, lineNumber):
        self.search = search
        self.name = name
        self.number = number
        self.lineNumber = lineNumber

    def searchError(self):                              #检索故障码
        global countNumber
        countNumber = 0
        for x in range(allLineNumber):    
            if self.search in allLine[x]:
                self.name[self.number] = allLine[x - 1]
                self.lineNumber[self.number] = x - 1
                self.number = self.number + 1 
                countNumber = self.number   

    def searchInformation(self):                        #检索信息值和信息类别
        global countNumber
        countNumber = 0
        for x in range(allLineNumber):
            if self.search in allLine[x]:
                self.name[self.number] = allLine[x]
                self.lineNumber[self.number] = x
                self.number = self.number + 1
                countNumber = self.number
    
    def searchInformation1(self):                        #检索其他需求信息
        global countNumber
        countNumber = 0
        for x in range(allLineNumber):
            if self.search in allLine[x]:
                self.lineNumber[self.number] = x
                self.number = self.number + 1
                countNumber = self.number

Failure = Foo('信息值： ', failure, failureNumber, failureLocation)
Failure.searchError()
failureNumber = countNumber
# print(failureNumber)

InformationValue = Foo('信息值： ', informationValue, informationValueNumber, informationValueLocation)
InformationValue.searchInformation()
informationValueNumber = countNumber
# print(informationValueNumber)

InformationCatefory = Foo('信息类别： ', informationCatefory, informationCateforyNumber, informationCateforyLocation)
InformationCatefory.searchInformation()
informationCateforyNumber = countNumber

DrivingObject = Foo("驱动对象： ", drivingObject, drivingObjectNumber, drivingObjectLocation)
DrivingObject.searchInformation1()
drivingObjectNumber = countNumber
# print(drivingObjectNumber)

Component = Foo('组件： ', component, componentNumber, componentLocation)
Component.searchInformation()
componentNumber = countNumber
# print(componentNumber)

Reason = Foo('原因： ', reason, reasonNumber,reasonLocation)
Reason.searchInformation1()
reasonNumber = countNumber

Processing = Foo('处理： ', processing, processingNumber, processingLocation)
Processing.searchInformation1()
processingNumber = countNumber

##提取驱动对象和组件
if drivingObjectNumber != componentNumber:
    print("信息提取有误！")
    print(drivingObjectNumber)
    print(componentNumber)
else:
    for x in range(drivingObjectNumber):
        ##提取组件
        component[x] = allLine[componentLocation[x]]
        ##提取驱动对象
        if drivingObjectLocation[x] == componentLocation[x] - 1:    #如果驱动对象只有一行
            drivingObject[x] = allLine[drivingObjectLocation[x]]
        else:
            dataDrivingObject = ''                                                           #如果驱动对象有多行
            lineRange = componentLocation[x] - drivingObjectLocation[x]
            for y in range(lineRange):    
                lineNumber = drivingObjectLocation[x] + y
                dataDrivingObject = dataDrivingObject + allLine[lineNumber]
                drivingObject[x] = dataDrivingObject

##提取原因和处理
if reasonNumber != processingNumber:
    print("信息提取有误！")
    print(reasonNumber)
    print(processingNumber)
elif reasonNumber != failureNumber:
    print("信息提取有误！")
else:
    for x in range(reasonNumber):
        ##提取原因
        if reasonLocation[x] == processingLocation[x] - 1:          #如果原因只有一行
            reason[x] = allLine[reasonLocation[x]]
        else:
            dataReason = ''                                                           #如果原因有多行
            lineRange = processingLocation[x] - reasonLocation[x]
            for y in range(lineRange):
                lineNumber = reasonLocation[x] + y
                dataReason = dataReason + allLine[lineNumber]
                reason[x] = dataReason
        ##提取处理
        if x != processingNumber - 1:                                           #如果处理的不是最后一条
            if processingLocation[x] == failureLocation[x + 1] - 1:     #如果处理只有一行
                processing[x] = allLine[processingLocation[x]]
            else: 
                dataProcess = ''                                                              #如果处理有多行
                lineRange = failureLocation[x + 1] - processingLocation[x]
                for y in range(lineRange):
                    lineNumber = processingLocation[x] + y
                    dataProcess = dataProcess + allLine[lineNumber]
                    processing[x] = dataProcess
        else:
            dataProcess = ''
            lineRange = allLineNumber - processingLocation[x]
            for y in range(lineRange):   
                lineNumber = processingLocation[x] + y
                dataProcess = dataProcess + allLine[lineNumber]
                processing[x] = dataProcess

##将关键信息转存到EXCEL
excelPath = "./S120_failure_code_table.xlsx"

def CreateExcel(information1, information2, information3, information4, information5, information6, information7, path):
    data = {'故障名称':information1, '信息值':information2, '信息类别':information3, '驱动对象':information4, '组件':information5, '原因':information6, '处理':information7}
    df = pd.DataFrame(data)
    df.to_excel(path) 

CreateExcel(failure, informationValue, informationCatefory, drivingObject, component, reason, processing, excelPath)

print("Done!")

Done!


### 文本内容及其他信息提取（用于主程序）
##### 各关键词检索的文本内容、总数、位置均已记录

In [3]:
txtPath = './TXT/S120_failure_code_list.txt'
## 获取每行的信息和内容
with open(txtPath, 'r', encoding = 'utf-8') as f:
    allLine = f.readlines()
    f.close()
allLineNumber = len(allLine)
# print(allLineNumber)

## 剔除不需要的信息行
with open(txtPath,"w",encoding="utf-8") as f_w:
    for line in allLine:
        if "故障和报警" in line:
            continue
        if "SINAMICS S120/S150" in line:
            continue
        if "参数手册," in line:
            continue
        f_w.write(line)

## 刷新每行的信息和内容
with open(txtPath, 'r', encoding = 'utf-8') as f:
    allLine = f.readlines()
    f.close()
allLineNumber = len(allLine)
# print(allLineNumber)

## 筛选分类文本信息
failure = {}                #故障码和名称
failureNumber = 0
failureLocation = {}

informationValue = {}           #信息值
informationValueNumber = 0
informationValueLocation = {}

informationCatefory = {}        #信息类别
informationCateforyNumber = 0
informationCateforyLocation = {}

drivingObject = {}
drivingObjectNumber = 0         #驱动对象数量
drivingObjectLocation = {}    #驱动对象所在首行

component = {}
componentNumber = 0             #组件数量 
componentLocation = {}        #组件所在行

reason = {}
reasonNumber = 0                #原因数量
reasonLocation = {}           #原因所在首行

processing = {}
processingNumber = 0            #处理数量
processingLocation = {}       #处理所在首行

#####################
###基于常规方法编写###
#####################
for x in range(allLineNumber):
    if "信息值： " in allLine[x]:
        # print(allLine[i])
        failure[failureNumber] = allLine[x - 1]                 #故障码和名称
        failureLocation[failureNumber] = x - 1                  #故障码所在行
        informationValue[failureNumber] = allLine[x]            #信息值
        informationCatefory[failureNumber] = allLine[x + 1]     #信息类别
        failureNumber = failureNumber + 1
    if "驱动对象： " in allLine[x]:
        drivingObjectLocation[drivingObjectNumber] = x
        drivingObjectNumber = drivingObjectNumber + 1
    if "组件： " in allLine[x]:
        componentLocation[componentNumber] = x
        componentNumber = componentNumber + 1
    if "原因： " in  allLine[x]:
        reasonLocation[reasonNumber] = x
        reasonNumber = reasonNumber + 1
    if "处理： " in allLine[x]:
        processingLocation[processingNumber] = x
        processingNumber = processingNumber + 1

##提取驱动对象和组件
if drivingObjectNumber != componentNumber:
    print("信息提取有误！")
    print(drivingObjectNumber)
    print(componentNumber)
else:
    for x in range(drivingObjectNumber):
        ##提取组件
        component[x] = allLine[componentLocation[x]]
        ##提取驱动对象
        if drivingObjectLocation[x] == componentLocation[x] - 1:        #如果驱动对象只有一行
            drivingObject[x] = allLine[drivingObjectLocation[x]]
        else:
            dataDrivingObject = ''                                      #如果驱动对象有多行
            lineRange = componentLocation[x] - drivingObjectLocation[x]
            for y in range(lineRange):    
                lineNumber = drivingObjectLocation[x] + y
                dataDrivingObject = dataDrivingObject + allLine[lineNumber]
                drivingObject[x] = dataDrivingObject

##提取原因和处理
if reasonNumber != processingNumber:
    print("信息提取有误！")
    print(reasonNumber)
    print(processingNumber)
elif reasonNumber != failureNumber:
    print("信息提取有误！")
else:
    for x in range(reasonNumber):
        ##提取原因
        if reasonLocation[x] == processingLocation[x] - 1:              #如果原因只有一行
            reason[x] = allLine[reasonLocation[x]]
        else:
            dataReason = ''                                             #如果原因有多行
            lineRange = processingLocation[x] - reasonLocation[x]
            for y in range(lineRange):
                lineNumber = reasonLocation[x] + y
                dataReason = dataReason + allLine[lineNumber]
                reason[x] = dataReason
        ##提取处理
        if x != processingNumber - 1:                                   #如果处理的不是最后一条
            if processingLocation[x] == failureLocation[x + 1] - 1:     #如果处理只有一行
                processing[x] = allLine[processingLocation[x]]
            else: 
                dataProcess = ''                                        #如果处理有多行
                lineRange = failureLocation[x + 1] - processingLocation[x]
                for y in range(lineRange):
                    lineNumber = processingLocation[x] + y
                    dataProcess = dataProcess + allLine[lineNumber]
                    processing[x] = dataProcess
        else:
            dataProcess = ''
            lineRange = allLineNumber - processingLocation[x]
            for y in range(lineRange):   
                lineNumber = processingLocation[x] + y
                dataProcess = dataProcess + allLine[lineNumber]
                processing[x] = dataProcess

'''切割故障码和故障名称'''
###输入存有故障码和名称的词典failure以及故障码总数failureNumber
###输出分别存有故障码和故障名的两个词典code和name
def cutFailure(information, number):
    code = {}
    name = {}

    for n in range(number):
        str = information[n]
        str2 = str.split(' ', 1)
        code[n] = str2[0]
        name[n] = str2[1]

    return code, name

failureCode, failureName = cutFailure(failure, failureNumber)

########################
##提取指定故障码的相关信息（提取单个故障码）
########按钮功能#########
########################
###输入故障码词典、故障码数量、故障码所在行位置词典、要检索的故障码
###输出被检索的故障码和相关信息
def GetFailureInformation(codeList, number, location, targetCode):
    missionComplete = False     #故障码查询结果标志位
    targetCount = 0             #故障码计数
    if len(targetCode) != 6:
        missionComplete = False
    else:   
        for m in range(number):
            if targetCode in codeList[m]:
                targetNumber = m
                missionComplete = True
                print(targetNumber)           

    if missionComplete == True:    
        targetLocationUp = location[targetNumber]
        targetLocationDown = location[targetNumber + 1]
        targetRange = targetLocationDown - targetLocationUp

        dataTarget = ''
        for n in range(targetRange):
            lineNumber = targetLocationUp + n
            dataTarget = dataTarget + allLine[lineNumber]
        return dataTarget
    else:
        missionFailed = '您输入的故障码有误，请核验后再次输入！\n'
        return missionFailed        

searchingCode = 'F07011'
GetFailureInformation(failureCode, failureNumber, failureLocation, searchingCode)

440
441


'F07011 驱动：电机超温\n信息值： %1\n信息类别： 电机过载 (8)\n驱动对象： VECTOR, VECTOR_AC, VECTOR_I_AC\n组件： 电机 传播： GLOBAL\n反应： OFF2 (OFF1, OFF3, STOP2, 无)\n应答： 立即\n原因： KTY84/PT1000 或没有传感器：\n测得的电机温度或者模型温度超过了故障阈值(p0605)，或者在超过报警阈值(p0604)之后延迟时间(p0606)届满。\n执行 p0610 中设置的反应。\nPTC 或者双金属常闭触点：\n超过了 1650 欧姆的动作阈值，或者常闭触点打开，而且延迟时间(p0606)已届满。执行 p0610 中设置的反应。\n可能的原因：\n-电机超载。\n-电机环境温度过高。\n- PTC/双金属常闭触点：断线或未连接传感器。\n故障值（r0949，十进制）：\n200:\n电机温度模型 1 (I2t)：温度过高。\n选择了 SME 或 TM120 (p0601 = 10, 11) 时，故障值为触发该信息的温度通道号。\n参见： p0351, p0604, p0605, p0606, p0612, p0613, p0617, p0618, p0619, p0625, p0626, p0627, p0628\n处理： -减小电机负载。\n-检查环境温度和电机通风。\n-检查 PTC 或双金属常闭触点的布线和连接。\n参见： p0351, p0604, p0605, p0606, p0612, p0617, p0618, p0619, p0625, p0626, p0627, p0628\n'